In [1]:
import pyodbc
import pandas as pd

server = "localhost,5432"
db_name = "synpuf"
driver = "/usr/local/lib/psqlodbcw.so"

cnxn = pyodbc.connect("DRIVER={" + driver + "};SERVER=" + server + ";DATABASE=" + db_name + ";Trusted_Connection=yes;")

In [20]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
person = pd.read_sql('SELECT * FROM person;', cnxn)
person.head()

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,1,8507,1923,5,1,None,8527,38003564,1,None,None,00013D2EFD8E45D1,1,None,1,None,1,None
1,2,8507,1943,1,1,None,8527,38003564,2,None,None,00016F745862898F,1,None,1,None,1,None
2,3,8532,1936,9,1,None,8527,38003564,3,None,None,0001FDD721E223DC,2,None,1,None,1,None
3,4,8507,1941,6,1,None,0,38003563,4,None,None,00021CA6FF03E670,1,None,5,None,5,None
4,5,8507,1936,8,1,None,8527,38003564,5,None,None,00024B3D2352D2D0,1,None,1,None,1,None


In [15]:
y = person.gender_concept_id
X = person[['year_of_birth', 'month_of_birth', 'day_of_birth', 'race_concept_id', 'ethnicity_concept_id', 'location_id', 'gender_source_value']]

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.5)

In [16]:
X.head()

,year_of_birth,month_of_birth,day_of_birth,race_concept_id,ethnicity_concept_id,location_id,gender_source_value
0,1923,5,1,8527,38003564,1,1
1,1943,1,1,8527,38003564,2,1
2,1936,9,1,8527,38003564,3,2
3,1941,6,1,0,38003563,4,1
4,1936,8,1,8527,38003564,5,1


In [17]:
model = svm.SVC()
model.fit(X_train, y_train)

SVC()

In [18]:
predicted_qualities = model.predict(X_test)

In [21]:
metrics = pd.DataFrame([["r2", r2_score(y_test, predicted_qualities)],
                        ["mse", mean_squared_error(y_test, predicted_qualities)],
                        ["mar", mean_absolute_error(y_test, predicted_qualities)]],
                columns=["metric", "value"])
metrics

,metric,value
0,r2,-0.879699
1,mse,292.500000
2,mar,11.700000
